In [17]:
# detect
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import hashlib


g = 8
seed = 10000
secret_key_1 = "123"
secret_key_2 = "456"
threshold = 3
watermark_information_length = 8

dataset='covertype'


original_file = '../dataset/covtype_with_key.subset.data'
origin = pd.read_csv(original_file)

np.random.seed(seed)

cover_types = origin['Cover_Type'].unique()
cover_types.sort()

def hash_mod(key, mod_value, secret_key):
    # 使用 SHA-256 生成哈希
    combined = f"{secret_key}{key}"
    hash_value = int(hashlib.sha256(combined.encode()).hexdigest(), 16)  # 将哈希转换为整数
    return hash_value % mod_value  # 对结果取模

columns_of_interest = ['Elevation', 'Aspect']  # 可根据需求更改
origin[columns_of_interest] = origin[columns_of_interest].fillna(0)  # 填充NA为0

# 提取前两位非零数字的函数
def first_two_digits(x):
    if x == 0:
        return "00"
    digits = str(x).lstrip('0.').replace('.', '')  # 去掉前导0和小数点
    if len(digits) == 1:  # 如果只有一位数字
        return digits + "0"  # 补0
    return digits[:2]  # 返回前两位

cover_types = origin['Cover_Type'].unique()
cover_types.sort()  

np.random.seed(seed)
shuffled_cover_types = list(cover_types)
shuffled_cover_types = sorted(shuffled_cover_types)
np.random.shuffle(shuffled_cover_types)
half_size = len(shuffled_cover_types) // 2
green_domain = shuffled_cover_types[:half_size]
red_domain = shuffled_cover_types[half_size:]


matches = []
data_path = "different_version_datasets/original"
for file_name in os.listdir(data_path):
    if file_name.startswith("covertype-"):
        # 提取 "-" 后的数字部分
        base_name, ext = os.path.splitext(file_name)
        leble_decimal_number = int(base_name.split("-")[-1])
        # print(f"leble_decimal_number: {leble_decimal_number}")
        # label_decimal_watermark_information.append(decimal_number)
        loaded_results = np.load(f"{data_path}/{file_name}", allow_pickle=True).item()
        watermarked_data = loaded_results['watermarked_data']

        detected_watermark_information = ""
        watermarked_data[columns_of_interest] = watermarked_data[columns_of_interest].fillna(0)  # 填充NA为0


        green_cells = np.zeros(watermark_information_length)
        n_cells = np.zeros(watermark_information_length)
        z_scores = np.zeros(watermark_information_length)

        for idx in range(len(watermarked_data)):

            selected_data = watermarked_data.loc[idx, columns_of_interest]
            first_two_digits_data = selected_data.apply(first_two_digits)
            composite_numbers = ''.join(first_two_digits_data.values)
            
            w_index = hash_mod(composite_numbers, watermark_information_length, secret_key_1)
            
            if(hash_mod(composite_numbers, g, secret_key_2) == 0): 
                n_cells[w_index] += 1    
                if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
                    green_cells[w_index] += 1
               
            
        for idx in range(watermark_information_length):
            if n_cells[idx] != 0:
                z_scores[idx] = (green_cells[idx] - n_cells[idx]/2 ) / math.sqrt(n_cells[idx]/4)
            else:
                z_scores[idx] = 0

        for idx in range(len(z_scores)):
            if z_scores[idx] > threshold:
                detected_watermark_information += '1'
            else:
                detected_watermark_information += '0'
        # print(f"Detected watermark information: {detected_watermark_information}")
        # print("----")
        if int(detected_watermark_information, 2) == leble_decimal_number:
            matches.append(1)
        else:
            matches.append(0)
        
print(f"Accuracy: {sum(matches) / len(matches)}")
        
        





Accuracy: 1.0


In [20]:
# detect
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import hashlib


g = 8
seed = 10000
secret_key_1 = "123"
secret_key_2 = "456"
threshold = 3
watermark_information_length = 8

dataset='covertype'


original_file = '../dataset/covtype_with_key.subset.data'
origin = pd.read_csv(original_file)

np.random.seed(seed)

cover_types = origin['Cover_Type'].unique()
cover_types.sort()

def hash_mod(key, mod_value, secret_key):
    # 使用 SHA-256 生成哈希
    combined = f"{secret_key}{key}"
    hash_value = int(hashlib.sha256(combined.encode()).hexdigest(), 16)  # 将哈希转换为整数
    return hash_value % mod_value  # 对结果取模

columns_of_interest = ['Elevation', 'Aspect']  # 可根据需求更改
origin[columns_of_interest] = origin[columns_of_interest].fillna(0)  # 填充NA为0

# 提取前两位非零数字的函数
def first_two_digits(x):
    if x == 0:
        return "00"
    digits = str(x).lstrip('0.').replace('.', '')  # 去掉前导0和小数点
    if len(digits) == 1:  # 如果只有一位数字
        return digits + "0"  # 补0
    return digits[:2]  # 返回前两位

cover_types = origin['Cover_Type'].unique()
cover_types.sort()  

np.random.seed(seed)
shuffled_cover_types = list(cover_types)
shuffled_cover_types = sorted(shuffled_cover_types)
np.random.shuffle(shuffled_cover_types)
half_size = len(shuffled_cover_types) // 2
green_domain = shuffled_cover_types[:half_size]
red_domain = shuffled_cover_types[half_size:]


attack_proportion = [0.2, 0.4, 0.6, 0.8, 1.0]
for proportion in attack_proportion:
    matches = []
    data_path = f"different_version_datasets/{proportion}attack"
    for file_name in os.listdir(data_path):
        if file_name.startswith("covertype-"):
            # 提取 "-" 后的数字部分
            base_name, ext = os.path.splitext(file_name)
            leble_decimal_number = int(base_name.split("-")[-1])
            # print(f"leble_decimal_number: {leble_decimal_number}")
            # label_decimal_watermark_information.append(decimal_number)
            watermarked_data = pd.read_csv(f"{data_path}/{file_name}")

            detected_watermark_information = ""
            watermarked_data[columns_of_interest] = watermarked_data[columns_of_interest].fillna(0)  # 填充NA为0


            green_cells = np.zeros(watermark_information_length)
            n_cells = np.zeros(watermark_information_length)
            z_scores = np.zeros(watermark_information_length)

            for idx in range(len(watermarked_data)):

                selected_data = watermarked_data.loc[idx, columns_of_interest]
                first_two_digits_data = selected_data.apply(first_two_digits)
                composite_numbers = ''.join(first_two_digits_data.values)
                
                w_index = hash_mod(composite_numbers, watermark_information_length, secret_key_1)
                
                if(hash_mod(composite_numbers, g, secret_key_2) == 0): 
                    n_cells[w_index] += 1    
                    if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
                        green_cells[w_index] += 1
                
                
            for idx in range(watermark_information_length):
                if n_cells[idx] != 0:
                    z_scores[idx] = (green_cells[idx] - n_cells[idx]/2 ) / math.sqrt(n_cells[idx]/4)
                else:
                    z_scores[idx] = 0

            for idx in range(len(z_scores)):
                if z_scores[idx] > threshold:
                    detected_watermark_information += '1'
                else:
                    detected_watermark_information += '0'
            # print(f"Detected watermark information: {detected_watermark_information}")
            # print("----")
            if int(detected_watermark_information, 2) == leble_decimal_number:
                matches.append(1)
            else:
                matches.append(0)
            
    # print(f"Accuracy: {sum(matches) / len(matches)}")
    print(f"Proportion: {proportion}, Accuracy: {sum(matches) / len(matches)}")
        
        





Proportion: 0.2, Accuracy: 1.0
Proportion: 0.4, Accuracy: 1.0
Proportion: 0.6, Accuracy: 0.5078125
Proportion: 0.8, Accuracy: 0.0
Proportion: 1.0, Accuracy: 0.0
